In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

# download data from browser
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100
).split_documents(docs)


/Users/lizylin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# option!! use huggungface embedding

from langchain.embeddings import HuggingFaceEmbeddings

#param encode_kwargs: Dict[str, Any] [Optional]
#Keyword arguments to pass when calling the encode method of the Sentence Transformer model, such as prompt_name, prompt, batch_size, precision, normalize_embeddings, and more. See also the Sentence Transformer documentation: https://sbert.net/docs/package_reference/SentenceTransformer.html#sentence_transformers.SentenceTransformer.encode

#param model_kwargs: Dict[str, Any] [Optional]¶
#Keyword arguments to pass to the Sentence Transformer model, such as device, prompts, default_prompt_name, revision, trust_remote_code, or token. See also the Sentence Transformer documentation: https://sbert.net/docs/package_reference/SentenceTransformer.html#sentence_transformers.SentenceTransformer

#param multi_process: bool = False
#Run encode() on multiple GPUs.

#param show_progress: bool = False
#Whether to show a progress bar.

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

In [4]:
# this will open a folder named aoai_chroma_db3 and create a sqlite file 

persist_directory = 'chroma_langchain_aoai_retrieval_agent'
vectordb = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()


retriever = vectordb.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

In [5]:
%run env_config_setting.ipynb

deploy_name = "gpt-35-turbo-16k"

api_key = openai.api_key
api_base = openai.api_base
api_version = openai.api_version

In [6]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import AzureChatOpenAI

client = AzureChatOpenAI(
    deployment_name=deploy_name,
    temperature=0,
    api_key=api_key,
    azure_endpoint=api_base,
    openai_api_version=api_version)


agent_chain = initialize_agent(
    [retriever_tool],
    llm=client,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

/Users/lizylin/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/Users/lizylin/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
# run the agent
agent_chain.run(
    "What are the Neurons Lab services",
)

/Users/lizylin/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




> Entering new AgentExecutor chain...
I can use the neurons_lab_search tool to find information about the Neurons Lab services. Let me do that for you.
Action:```{"action": "neurons_lab_search", "action_input": {"query": "Neurons Lab services"}}```
Observation: Privacy Notice
 Cookie Notice




[email protected]
 +44 330 027 2146



International House, 64 Nile Str,
London, N1 7SR, United Kingdom











© 2024 Neurons Lab LTD. Registered in England and Wales, 
Company Number 12265479
Createdby twid

Neurons Lab | Home


























 



Your vision
Our expertise
Secret sauce*


                            Recipe                            



Partner with us


                                for INNOVATION                                                    
We are AI solution development experts for fast-growing companies seeking innovation through collaboration.






















HealthTech
CleanTech
RetailTech

Yes, I am interested











Get in touch








'The Neurons Lab offers a range of services, including:\n\n1. AI feasibility analysis\n2. AI solution engineering\n3. AI operations management\n4. Generative AI Solutions\n\nThese services are aimed at fast-growing companies seeking innovation through collaboration. The lab specializes in HealthTech, CleanTech, and RetailTech solutions.\n\nIs there anything specific you would like to know about these services?'

In [8]:
# add yourself tools (defined function)

#use the @tooldecorator to make Langchain know about it
#describe function comment between """ """

from langchain.tools import BaseTool, StructuredTool, tool

@tool
def calculate_length_tool(a: str) -> int:
    """The function calculates the length of the input string."""
    return len(a)


In [9]:
from langchain.agents import initialize_agent

agent_chain2 = initialize_agent(
    [retriever_tool, calculate_length_tool],
    llm=client,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [10]:
agent_chain.run(
    "Get the length of user input",
)



> Entering new AgentExecutor chain...
To get the length of user input, you can use the `len()` function in Python. Here's an example:

```python
user_input = input("Enter something: ")
length = len(user_input)
print("Length of user input:", length)
```

This code prompts the user to enter something, calculates the length of the input using `len()`, and then prints the length.

> Finished chain.


'To get the length of user input, you can use the `len()` function in Python. Here\'s an example:\n\n```python\nuser_input = input("Enter something: ")\nlength = len(user_input)\nprint("Length of user input:", length)\n```\n\nThis code prompts the user to enter something, calculates the length of the input using `len()`, and then prints the length.'